In [1]:
#basic package
import json
import tqdm
import os
import glob
import pandas as pd
import sys
import shutil
import time
import re
import pickle

#to match substring in string
import fuzzysearch
from fuzzysearch import find_near_matches

#access structured data from pdf
import PyPDF2
from docx import *

In [2]:
#root directory of the project
ROOT_DIR = os.path.abspath("../")
path_data = os.path.join(ROOT_DIR,'datasets/synonyms/synonym_book')

In [3]:
PACKAGE_PARENT = '../../..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

from UTILS.utils import from_string_without_whitespace_to_string_withwithespace, extract_bold_text, from_chapter_to_structured_data

/home/camille/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/camille/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
#using quick pdf converter (free trial) to convert from pdf to doxc. But only until page 90+28 car si non must 
#buy..... TODO convert all
#this notebook is meant to be used on a file that is in doxc and in pdf

In [5]:
#strategy:
#PyPDF2 is reading the text in the proper order, while python-doxc not. Hence we will use the first one to read text,
#and the second to help for splitting. Indeed, unfortunately we can not use the result from pyhton-doxc 
#(text_no_whitespace_bold) to build a skeleton and then complet, as the order is wrong.

#main steps:
#1. extract needed data: extract bold-text using python-doxc. create a list of titles and subtitles (bold-data)
#2. Using PyPDF2, extract all the text (text-data)
#3. take each title and look for closest match in the text, creating  anew list of matched titles (Levinshtein dist.)
#4. split the text-data using the list of matched title (dataframe: title, info columns)
#5. split the info using subtitles as properties, synonyms etc

In [6]:
#Note:
#1.for each book such a notebook is necessary. (what makes it hard to automate without mistake: 
#mistake in the books cf bold, different pattern in each book and eventually chapter,...)

In [7]:
#some issues explaining why we need 3.:
#when downloaded from PyPDF2 there is '&' while downloading from python-doxc their is no '&' at least in some e.g. :
#from text: 1838Œ1849 (Atractaspididae)    from bold: 1838–1849(Atractaspididae)
#from text: Wüster & Broadley              from bold: WüsterBroadley
#from text: pl. 2, fig. a                  from bold: pl.fig.a
#--> we need to use rules as maximum number of caractere that need to be changed to say their ias a match

In [8]:
#hypothesis:
#bold text is not split within two pages. (otherwise will be problematique)
#no two consecutive title or subtitle (always synonyms, or types etc in between)

# Get necessary data

In [9]:
book_title = '2014 Snakes of the world-A catalogue of living and extinct species.pdf'
file_path = os.path.join(path_data,book_title)

### extract bold text using python-doxc

In [10]:
#document = Document('/home/camille/vm_exchange/Lab/snakes/datasets/synonyms/synonym_book/2014 Snakes of the world-A catalogue of living and extinct species - Copie - Copie.docx')
document = Document('/home/camille/vm_exchange/Lab/snakes/datasets/synonyms/synonym_book/2014 Snakes of the world-A catalogue of living and extinct species - chapterA.docx')

In [11]:
#extract bold text
r = extract_bold_text(document)
text_no_whitespace_bold = r.split('Valid Genera and Species')[-1].strip()    

### extract information needed for our specific book

In [12]:
#find properties 
pattern = re.compile(r'(Type species:|Distribution:|Sources:|Remarks:|Type locality:|Type:|Types:|Sources:|Source:|Synonym:|Synonyms:)')
li_text_no_whitespace_bold = pattern.split(text_no_whitespace_bold)
li_properties = list(set([i.strip() for i in li_text_no_whitespace_bold if (i.strip().endswith(':')) & (len(i)<25)]))
#in the book they forgot to put in bold after the properties: comment. hence we need to add it manually
li_properties = li_properties+['Comment:','Comments:']
# create pattern for properties
li_properties
pattern_properties = ''
for p in li_properties:
    pattern_properties = pattern_properties+'|'+p
pattern_properties = pattern_properties.strip('|')
pattern_properties

'Type locality:|Synonym:|Type:|Fossil records:|Synonyms:|Sources:|Remarks:|Type species:|Types:|Distribution:|Source:|Comment:|Comments:'

In [13]:
#create list of titles and list of subtitles
li = list(set([i.strip() for i in li_text_no_whitespace_bold if i not in li_properties]))
#python-doxc reading sometimes the index in middle of title
pattern = re.compile(r'[A-Z]{1}  ')
li = [j.strip() for i in li for j in pattern.split(i)]
#due to comment mistake:
pattern = re.compile(r'Comment  |Comments  ')
li = [j.strip() for i in li for j in pattern.split(i)]
li = [i for i in li if len(i)>2]
li_st = [i.strip() for i in li if i[0].isdigit()]
li_t = [i.strip() for i in li if (i[0].isupper()) & (i[1].isupper()) & (i[1].isupper())]

In [14]:
print(len(li_st+li_t))

534


### extract all text using PyPDF2

In [ ]:
pdf_file = open(os.path.join(path_data,book_title),'rb')
read_pdf = PyPDF2.PdfFileReader(pdf_file)
number_of_pages = read_pdf.getNumPages()
print('There is %d page in this pdf'%number_of_pages)

There is 1257 page in this pdf


page = read_pdf.getPage(491+28)   #(159)#(3+28)
page_content = page.extractText()
page_content.split('\n')

# Put info into structured data

In [ ]:
#for this specific book: add all page text info of a chapter, then create strucutred data for this chapter and
#continue with the next one
chapter_content = ''
for i in range(3+28,811+28):
    
    #print(i)
    page = read_pdf.getPage(i)
    page_content = page.extractText()
    li_page = page_content.split('\n')
    li_page_clean = li_page.copy()
    
    ### if empty page pass it ###
    if len(li_page)<2:
        pass 
    
    ### if in middle of chapter ###
    elif 'Snakes of the World' in li_page[0]:
        page_nbr = li_page[0].split('Snakes of the World')[0]
        
        #remove the page if it starts with the number, otherwise it means that its a new paragraph (i.e. not having 
        #the page written on the top...but then it should not have snakes of the world written)
        if li_page[1].startswith(page_nbr):
            first_element = li_page[1].split(page_nbr)[-1]
            li_page_clean = [first_element] + li_page[2:]
        else:
            print('snakes of the world but no new paragraph weird! check on book page %s'%(i-28))
        #add info of this page to the chapter
        chapter_content = chapter_content + ' ' + ' '.join(li_page_clean)
    elif 'Snakes of the World' in li_page[1]:    
        page_nbr = li_page[0]
        if li_page[0]!=li_page[2]:
            print('weird! show page %d'%i)
        li_page_clean = li_page[3:]
        
    ### if new paragraph, remove page and keep rest as title ### 
    else:
        #if not first paragraph, register all info you have of the paragraph from before
        if chapter_content!='':
            start_time = time.time()
            R, title_not_matched, li_matched_title, li_distance = from_chapter_to_structured_data(text=chapter_content, 
                                                                                                  li_title=li_t+li_st)
            print("--- %s seconds ---" % round(time.time() - start_time, 2))
            li_title = [R[i] for i in range(len(R)) if i%2==0]
            li_info = [R[i] for i in range(len(R)) if i%2!=0]
            print(len(li_title[0:-1]), len(li_info))
            l = max(len(li_title), len(li_info))
            df = pd.DataFrame.from_dict({'title':li_title[0:l], 'info':li_info[0:l]})
            df.to_csv(os.path.join(path_data,'df_info_chapter_'+title+'.csv'), index=False, sep=';')
            pickle.dump(title_not_matched, open(os.path.join(path_data,'li_title_notmatched_'+title+'.pkl'), 'wb'))
            pickle.dump(li_matched_title, open(os.path.join(path_data,'li_matched_title_'+title+'.pkl'), 'wb'))
            pickle.dump(li_distance, open(os.path.join(path_data,'li_distance_'+title+'.pkl'), 'wb'))

        #otherwise, pass to the next one
        #remove any digits from the first element of the page
        first_element = (' '.join(li_page[0:2]).lstrip('0123456789')).strip().strip('ƒ')
        #print(first_element, li_page[0], li_page[0].split(first_element))
        page_nbr = li_page[0].split(first_element)[0]
        title = first_element[0]
        print('-------------------paragraph: %s'%title)
        first_element = first_element[1:]
        li_page_clean = [first_element] + li_page[2:]
        #add info of this page to the chapter
        chapter_content = ' '.join(li_page_clean)
        

-------------------paragraph: A
ANOMOCHILUS Berg, 1901  (nomen substitutum) (Anomochilidae)
ARCHELAPHE Schulz, Böhme &amp; Tillack, 2011 (Colubridae)
APLOPELTUR A A.-M.-C. Duméril, 1853 (nomen protectum) (Pareatidae)
AUSTRELAPS Worre ll, 196 3a (Elapidae)
ARIZONA Kennicott in Baird, 1859a (Colubridae)
ATRACTASPIS A. Smith, 1849 in 1838–1849 (Atractaspididae)
AGKISTRODON Palisot de Beauvois, 1799
ASTROTIA J.G. Fischer, 1855 (Elapidae)
APARALLACTUS A. Smith, 1849 in 1838–1849 (Atractaspididae)
ANTAIOSERPENS Wells &amp; Wellington, 1985 (Elapidae)
ASPIDITES W.C.H. Peters, 1877a (nomen substitutum) (Pythonidae)
ADELOPHIS Dugès in Cope, 1879 (Natricidae)
ASPIDOMORPHUS Fitzinger, 1843 (Elapidae)
ACRANTOPHIS Jan, 1860 in JanSordelli, 1860–1866 (Boidae)
AMPLORHINUS A. Smith, 1847 in 1838–1849      42. Amphiesma viperinum (Schenkel, 1901). Verh. Naturf. Ges. Basel (1901–1902) 13(1): 155–156. (Xenochrophis viperinus)
ANTARESIA Wells &amp; Wellington, 1984 (Pythonidae)
ATRETIUM Cope, 1861e (nomen

start_time = time.time()
R = from_chapter_to_structured_data(text=chapter_content, li_title=li_t+li_st)
print("--- %s seconds ---" % round(time.time() - start_time, 2))

#examples for find near match
title = 'AFROTYPHLOPSBroadleyWallach,2009(Typhlopidae)'
text = ''.join(chapter_content.split(' '))
r = find_near_matches(title, text, max_l_dist=2, max_deletions=max(int(0.05*len(title)),1), 
                      max_insertions=1, max_substitutions=0)
print(r)
new_title = text[r[0][0]:r[0][1]]
new_title